## MITgcm Example 

xgcm is developed in close coordination with the [xmitgcm](http://xmitgcm.readthedocs.io/) package.
The metadata in datasets constructed by xmitgcm should always be compatible with xgcm's expectations.
xmitgcm is necessary for reading MITgcm's binary MDS file format.
However, for this example, the MDS files have already been converted and saved as netCDF.

Below are some example of how to make calculations on mitgcm-style datasets using xgcm.

First we import xarray and xgcm:

In [ ]:
import xarray as xr
import numpy as np
import xgcm
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10,6)

Now we open the example dataset, which is stored with the xgcm github repository in the datasets folder.

In [ ]:
# hack to make file name work with nbsphinx and binder
import os
fname = '../datasets/mitgcm_example_dataset_v2.nc'
if not os.path.exists(fname):
    fname = '../' + fname
    
ds = xr.open_dataset(fname)
ds

### Creating the grid object

Next we create a `Grid` object from the dataset.
We need to tell xgcm that the `X` and `Y` axes are periodic.
(The other axes will be assumed to be non-periodic.)

In [ ]:
grid = xgcm.Grid(ds, periodic=['X', 'Y'])
grid

We see that xgcm identified five different axes: X (longitude), Y (latitude), Z (depth),  T (time), and 1RHO (the axis generated by the output of the LAYERS package).

### Divergence Calculation

Here we show how to calculate the divergence of the volume transport.
The precise details of how to do this calculation are highly model- and configuration-dependent (e.g. free-surface vs. rigid lid, etc.)
In this very simple example, the flow is incompressible, satisfying the continuity equation

$$ \frac{\partial u}{\partial x} + \frac{\partial v}{\partial y} + \frac{\partial w}{\partial z} = 0 \ .$$

In discrete form, using [MITgcm notation](http://mitgcm.org/public/r2_manual/latest/online_documents/node50.html), the equation becomes

$$ \delta_i \Delta y_g \Delta r_f h_w u + \delta_j \Delta x_g \Delta r_f h_s v + \delta_k A_c w = \delta_k A_c (E - P)_{r=0} $$

where we have added the volume source due to evaporation minus precipitation at the sea surface on the RHS.

First we calculate the divergence of the horizontal flow only (u and v), i.e. the first two terms on the LHS. To begin, we multiply the u and v velocity components by the appropriate area factors to convert them to volume transports.

In [ ]:
u_transport = ds.UVEL * ds.dyG * ds.hFacW * ds.drF
v_transport = ds.VVEL * ds.dxG * ds.hFacS * ds.drF

The `u_transport` DataArray is on the left point of the X axis, while the `v_transport` DataArray is on the left point of the Y axis.

Now comes the xgcm magic: we take the diff along both axes and divide by the cell area element to find the divergence of the horizontal flow. Note how this new variable is at the cell center point.

In [ ]:
div_uv = (grid.diff(u_transport, 'X') + grid.diff(v_transport, 'Y'))
div_uv

To verify that this worked properly, we can compute the vertical sum of the horizontal convergence.
It should be equal to the vertical velocity at the top.

In [ ]:
(-div_uv.sum(dim='Z') / ds.rA).plot();

In [ ]:
ds.WVEL[0,0].plot();

Indeed these two fields look visually identical, which we can verify with a scatter plot.

In [ ]:
plt.scatter((-div_uv.sum(dim='Z') / ds.rA), ds.WVEL[0,0], s=1)
plt.ylim([-1e-7, 1e-7])
plt.xlim([-1e-7, 1e-7])
plt.grid()

Finally, we can compute the full 3D divergence by including the vertical transport.

In [ ]:
w_transport = ds.WVEL * ds.rA
div_w = -grid.diff(w_transport, 'Z', boundary='fill')
div_w

Another scatterplot shows that the divergence of the horizontal velocity is equal and opposite to the divergence of the vertical velocity.

In [ ]:
plt.scatter(div_uv, -div_w, s=1)
plt.grid()

### Vorticity and Strain
Here we compute more dervied quantities from the velocity field.

The vertical component of the vorticity is a fundamental quantity of interest in ocean circulation theory. It is defined as

$$ \zeta = - \frac{\partial u}{\partial y} + \frac{\partial v}{\partial x} \ . $$

On the c-grid, a finite-volume representation is given by

$$ \zeta =  (- \delta_j \Delta x_c u + \delta_i \Delta y_c v ) / A_\zeta \ . $$

In xgcm, we calculate this quanity as

In [ ]:
zeta = (-grid.diff(ds.UVEL * ds.dxC, 'Y') + grid.diff(ds.VVEL * ds.dyC, 'X'))/ds.rAz
zeta

...which we can see is located at the `YG, XG` horizontal position (also commonly called the vorticity point).

We plot the vertical integral of this quantity, i.e. the barotropic vorticity:

In [ ]:
zeta_bt = (zeta * ds.drF).sum(dim='Z')
zeta_bt.plot(vmax=2e-4)

A different way to calculate the barotropic vorticity is to take the curl of the vertically integrated velocity.
This formulation also allows us to incorporate the $h$ factors representing partial cell thickness.

In [ ]:
u_bt = (ds.UVEL * ds.hFacW * ds.drF).sum(dim='Z')
v_bt = (ds.VVEL * ds.hFacS * ds.drF).sum(dim='Z')
zeta_bt_alt = (-grid.diff(u_bt * ds.dxC, 'Y') + grid.diff(v_bt * ds.dyC, 'X'))/ds.rAz
zeta_bt_alt.plot(vmax=2e-4)

Another interesting quantity is the horizontal strain, defined as

$$ s = \frac{\partial u}{\partial x} - \frac{\partial v}{\partial y} \ . $$

On the c-grid, a finite-volume representation is given by

$$ s =  (\delta_i \Delta y_g u - \delta_j \Delta x_g v ) / A_c \ . $$

In [ ]:
strain = (grid.diff(ds.UVEL * ds.dyG, 'X') - grid.diff(ds.VVEL * ds.dxG, 'Y')) / ds.rA
strain[0,0].plot()

### Barotropic Transport Streamfunction

We can use the barotropic velocity to calcuate the barotropic transport streamfunction, defined via

$$ u_{bt} = - \frac{\partial \Psi}{\partial y} \ , \ \ v_{bt} = \frac{\partial \Psi}{\partial x} \ .$$

We calculate this by integrating $u_{bt}$ along the Y axis using the grid object's `cumsum` method:

In [ ]:
psi = grid.cumsum(-u_bt * ds.dyG, 'Y', boundary='fill')
psi

We see that xgcm automatically shifted the Y-axis position from center (YC) to left (YG) during the cumsum operation.

We convert to sverdrups and plot with a contour plot.

In [ ]:
(psi[0] / 1e6).plot.contourf(levels=np.arange(-160, 40, 5))

### Kinetic Energy

Finally, we plot the kinetic energy $1/2 (u^2 + v^2)$ by interpoloting both quantities the cell center point.

In [ ]:
ke = 0.5*(grid.interp((ds.UVEL*ds.hFacW)**2, 'X') + grid.interp((ds.VVEL*ds.hFacS)**2, 'Y'))
ke[0,0].plot()